In [ ]:
import cupy as cp
import polars as pl
from gqimax.mapper import weightss_to_lambda
from gqimax.instructor import Instructor
from gqimax.utils import word_to_index, pauli_to_index, index_to_word, create_word_zj

class PStabilizers:
    def __init__(self, num_qubits: int):
        """PStabilizer is a Pauli term
        I encode it as two lists: indices (encoded Pauli words) and lambdas

        Args:
            j (int): index of stabilizer in the stabilizer group (generator)
            num_qubits (int)
        """
        # Init n stabilizer 
        # Each stabilizer is Z_j = I \otimes ... \otimes I \otimes Z (j^th) \otimes I \otimes ... \
        self.num_qubits = num_qubits
        # Lambdas for all stabilizers
        # It has the size n x 4^n
        self.lambdass = cp.ones(num_qubits)
        # Max size of indices is n x 4^n x n x 4
        self.indicess = cp.array([word_to_index(create_word_zj(j, num_qubits)) for j in range(num_qubits)])
        return
    def at(self, i: int, j: int | str):
        if type(j) == str:
            j = word_to_index(j)
        return self.lambdas[i][cp.where(self.indices == j)[0]]
    def map(self, ins: Instructor):
        for j, order in enumerate(ins.orders):
            k = j // 2
            if order == 0:
                self.indicess, self.lambdass = map_noncx(self.indicess, self.lambdass, ins.LUT, k, self.num_qubits)
            else:
                self.indicess, self.lambdass = map_cx(self.indicess, self.lambdass, ins.xoperators[k], self.num_qubits)
        return
    def __str__(self) -> str:
        text = ""
        lambdas_cp = self.lambdas.get()[:3]
        indices_cp = self.indices.get()[:3]
        for i, index in enumerate(indices_cp):
            text += (f"{cp.round(lambdas_cp[i], 2)} * {index_to_word(index, self.num_qubits)} + ")
        text += (f"... + {cp.round(self.lambdas.get()[-1], 2)} * {index_to_word(self.indices.get()[-1], self.num_qubits)}")
        return text
    


In [4]:
cp.array([word_to_index(create_word_zj(0, 4))])

array([192])